In [ ]:
import pandas as pd

corpus = pd.read_csv('quotes_classical_clean.csv', sep='|').sample(frac=1, random_state=0)

In [ ]:
import datasets

corpus_dataset = datasets.Dataset.from_pandas(corpus)
corpus_dataset

In [ ]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = encoder.encode(
    corpus_dataset["quote"],
    batch_size=100,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True,
)

In [ ]:
dset_embed = datasets.Dataset.from_dict({"embeddings": embeddings})
embeddings_dataset = datasets.concatenate_datasets([corpus_dataset, dset_embed], axis=1)
embeddings_dataset.add_faiss_index(column="embeddings")

In [ ]:
sentence = "Knowledge of history is power."
sentence_embedding = encoder.encode([sentence])
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", sentence_embedding, k=10
)
samples

In [ ]:
import pickle

with open('model.pickle', 'wb') as pkl:
    pickle.dump(encoder, pkl)
with open('embeddings_dataset.pickle', 'wb') as pkl:
    pickle.dump(embeddings_dataset, pkl)

# End of notebook